## Resized

In [4]:
import os
import cv2

def resize_and_save_images(source_dir, target_dir, resize_percentage=60):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    for folder_name in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder_name)
        if os.path.isdir(folder_path):
            print(f"Обработка папки: {folder_name}")
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    try:
                        
                        img = cv2.imread(file_path)
                        if img is None:
                            print(f"Не удалось прочитать файл {file_path}. Возможно, это не изображение.")
                            continue
                            
                        width = int(img.shape[1] * (resize_percentage / 100))
                        height = int(img.shape[0] * (resize_percentage / 100))

                        resized_img = cv2.resize(img, (width, height), interpolation=cv2.INTER_LANCZOS4)

                        new_file_name = f"{folder_name}_{file_name}"
                        output_path = os.path.join(target_dir, new_file_name)

                        cv2.imwrite(output_path, resized_img)
                        print(f"Сохранено как: {output_path}")
                    except Exception as e:
                        print(f"Ошибка при обработке файла {file_path}: {e}")

source_directory = "../data_anya/Les-2/"
target_directory = "../data_anya/resized-2/"

resize_and_save_images(source_directory, target_directory)

Обработка папки: .ipynb_checkpoints
Обработка папки: 4
Сохранено как: ../data_anya/resized-2/4_315.png
Сохранено как: ../data_anya/resized-2/4_104.png
Сохранено как: ../data_anya/resized-2/4_102.png
Сохранено как: ../data_anya/resized-2/4_308.png
Сохранено как: ../data_anya/resized-2/4_206.png
Сохранено как: ../data_anya/resized-2/4_103.png
Сохранено как: ../data_anya/resized-2/4_112.png
Сохранено как: ../data_anya/resized-2/4_213.png
Сохранено как: ../data_anya/resized-2/4_203.png
Сохранено как: ../data_anya/resized-2/4_101.png
Сохранено как: ../data_anya/resized-2/4_311.png
Сохранено как: ../data_anya/resized-2/4_313.png
Сохранено как: ../data_anya/resized-2/4_216.png
Сохранено как: ../data_anya/resized-2/4_109.png
Сохранено как: ../data_anya/resized-2/4_209.png
Сохранено как: ../data_anya/resized-2/4_113.png
Сохранено как: ../data_anya/resized-2/4_204.png
Сохранено как: ../data_anya/resized-2/4_215.png
Сохранено как: ../data_anya/resized-2/4_309.png
Сохранено как: ../data_anya/resiz

## Function for saving annotation

In [11]:
import os
import json

def add_annotation(image_path, caption, annotations_file):
    """
    Добавляет аннотацию для изображения в JSON-файл.

    :param image_path: Путь к изображению.
    :param caption: Текстовое описание (аннотация) для изображения.
    :param annotations_file: Путь к JSON-файлу с аннотациями.
    """
    if os.path.exists(annotations_file):
        with open(annotations_file, 'r', encoding='utf-8') as f:
            annotations = json.load(f)
    else:
        annotations = []

    annotations.append({
        "image_path": image_path,
        "caption": caption
    })

    with open(annotations_file, 'w', encoding='utf-8') as f:
        json.dump(annotations, f, ensure_ascii=False, indent=4)

    print(f"Аннотация для изображения '{image_path}' успешно добавлена.")

## Function for annotating

In [12]:
import requests
import json
import base64
from pathlib import Path


def annotating(image_path):
    def get_flower_image(image_path):
        with open(image_path, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
            return f"data:image/jpeg;base64,{encoded_string}"
            
    base64_image = get_flower_image(image_path)
    url = "https://openrouter.ai/api/v1/chat/"

    # Headers для запроса
    headers={
        "Authorization": "",
        "Content-Type": "",
        "HTTP-Referer": "", 
        "X-Title": "", 
      }
    
    # Тело запроса
    payload = {
        "model": "google/gemini-2.0-flash-001",
        "messages": [
        {
            "role": "system",
            "content": "You are an image recognition AI that provides detailed descriptions of images on russian ."
        },
        {
            "role": "user",
            "content": [
              {
                "type": "text",
                "text": '''Опиши траекторию перемещения взгляда по следующему шаблону:
                            
                            1. **Текст вопроса:** 
                            - Текст вопроса со слайда
                            
                            2. **Начало движения:**
                            - Где начинается движение взгляда? Например: «Взгляд начинает движение с первого слова вопроса: "Текст вопроса".»
                            
                            3. **Изменение диаметра зрачка:**
                            - На каких словах или фрагментах текста наблюдалось сужение зрачка (синий круг)?
                            - На каких словах или фрагментах текста наблюдалось расширение зрачка (красный круг)?
                            - На каких словах или фрагментах текста диаметр зрачка оставался средним (зеленый круг)?
                            
                            4. **Движение взгляда:**
                            - Опиши последовательность движения взгляда по тексту. Например: «Взгляд движется от начала вопроса вправо по строке, затем переходит ко второй строке.»
                            - Укажи реверсивные саккады (красные стрелки), если они есть.
                            - Укажи прямые саккады (синие стрелки), если они есть.
                            - Укажи переключение между ответами (зеленые стрелки), если оно наблюдалось.
                            
                            5. **Выбор ответа:**
                            - Какой ответ выбрал человек? Например: «Человек выбрал ответ "Да" после длительной фиксации на этом варианте.»
                            
                            **Важное правило:**  
                            - Запрещено начинать описание фразами типа: «Конечно, вот описание», «Давайте проанализируем траекторию», «Начнем с того, что» или любыми другими вводными фразами. Описание должно быть кратким и начинаться непосредственно с фактической информации.
                            - Очень важно правильно распознать какой ответ выбрал человек
                            - Используются только эти вопросы:
                            ['В настоящее время я проживаю на территории Российской Федерации?', 'Я в состоянии спокойно читать текст с экрана монитора компьютера вслух?', 'Я боюсь, что будет задан вопрос, на который не смогу ответить правдиво?', 
 'Я всегда в жизни настойчиво добиваюсь поставленной перед собой цели?', 'Я какой-либо материальный ущерб компании за время работы наносил?', 'Когда я сильно нервничаю, у меня усиливается частота сердцебиения?',
 'За последний год работы в компании, я получал незаконный доход?', 'В принципе, я способен на серьезный обман, если мне это будет выгодно?', 'Работая в компании за последний год, я трудовую дисциплину нарушал?',
 'Когда я обманываю, испытываю ли я неловкость и беспокойство?', 'За последние двенадцать месяцев я употреблял какие либо наркотики?', 'Меня беспокоят возможные серьезные неудачи в моей жизни и на работе?',
 'Я от руководства скрывал существенную информацию о работе?', 'У меня в жизни были неприятные ситуации, о которых не хочется вспоминать?', 'Мне достоверно известно о фактах какого- либо хищения в компании?', 
 'Я солгал на какой-либо из вопросов теста?']
                            
                            **Легенда:**
                            - Синий круг: Суженный зрачок. Обозначает, что во время фиксации зрачок сужался.
                            - Зеленый круг: Средний зрачок. Обозначает, что диаметр зрачка не изменялся во время фиксации.
                            - Красный круг: Расширенный зрачок. Обозначает, что зрачок расширялся во время фиксации.
                            - Красная стрелка: Реверсивная саккада. Обозначает движение взгляда, которое возвращается к предыдущей точке фиксации.
                            - Синяя стрелка: Прямая саккада. Обозначает прямое движение взгляда от одной точки фиксации к другой.
                            - Зеленая стрелка: Переключение между ответами. Обозначает переключение взгляда между различными вариантами ответов.
                            - Маленький круг: Короткая фиксация. Обозначает короткую по продолжительности фиксацию взгляда.
                            - Большой круг: Длительная фиксация. Обозначает длительную по продолжительности фиксацию взгляда.
                            - Выбор ответа осуществляется длительной фиксацией.'''
                },
                {
                        "type": "image_url",
                        "image_url": {
                            "url": base64_image  # Используем Base64-кодированное изображение
                        }
                    }
                ]
            }
        ]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        data = response.json()
    
        if "choices" in data and len(data["choices"]) > 0:
            assistant_message = data["choices"][0]["message"]["content"]
            return(assistant_message)
        else:
            print(f"Для {image_path} Не удалось найти описание в ответе.")
    else:
        print(f"Для {image_path} Ошибка: {response.status_code}")
        print(response.text)

In [ ]:
                "text": "Опиши траекторию перемещения взгляда по следующему шаблону:/n 1. **Начало движения:** /n   - Где начинается движение взгляда? Например: «Взгляд начинает движение с первого слова вопроса: "Текст вопроса".»/n 2. **Изменение диаметра зрачка:**/n  - На каких словах или фрагментах текста наблюдалось сужение зрачка (синий круг)?/n  - На каких словах или фрагментах текста наблюдалось расширение зрачка (красный круг)?/n   - На каких словах или фрагментах текста диаметр зрачка оставался средним (зеленый круг)? /n 3. **Движение взгляда:**/n  - Опиши последовательность движения взгляда по тексту. Например: «Взгляд движется от начала вопроса вправо по строке, затем переходит ко второй строке.»/n  - Укажи реверсивные саккады (красные стрелки), если они есть./n - Укажи прямые саккады (синие стрелки), если они есть./n  - Укажи переключение между ответами (зеленые стрелки), если оно наблюдалось./n 4. **Выбор ответа:**/n  - Какой ответ выбрал человек? Например: «Человек выбрал ответ "Да" после длительной фиксации на этом варианте.»/n 5. **Дополнительные наблюдения:** /n - Есть ли другие важные детали, такие как короткие фиксации (маленькие круги) или длительные фиксации (большие круги)? Например: «На слове "ключевое слово" наблюдалась длительная фиксация.»/n **Легенда:**/n- Синий круг: Суженный зрачок. Обозначает, что во время фиксации зрачок сужался./n- Зеленый круг: Средний зрачок. Обозначает, что диаметр зрачка не изменялся во время фиксации./n- Красный круг: Расширенный зрачок. Обозначает, что зрачок расширялся во время фиксации./n- Красная стрелка: Реверсивная саккада. Обозначает движение взгляда, которое возвращается к предыдущей точке фиксации./n - Синяя стрелка: Прямая саккада. Обозначает прямое движение взгляда от одной точки фиксации к другой./n- Зеленая стрелка: Переключение между ответами. Обозначает переключение взгляда между различными вариантами ответов./n - Маленький круг: Короткая фиксация. Обозначает короткую по продолжительности фиксацию взгляда./n - Большой круг: Длительная фиксация. Обозначает длительную по продолжительности фиксацию взгляда./n- Выбор ответа осуществляется длительной фиксацией."


## Test annotating

In [13]:
annotating("")

'1.  **Текст вопроса:**\n    - За последний год работы в компании, я получал незаконный доход?\n\n2.  **Начало движения:**\n    - Взгляд начинает движение с первого слова вопроса: "За"\n\n3.  **Изменение диаметра зрачка:**\n    - Сужение зрачка (синий круг): "За", "последний", "год", "в", "компании", "я"\n    - Расширение зрачка (красный круг): "работы", "незаконный", "доход", "Нет"\n    - Средний зрачок (зеленый круг): "последний", "год", "работы", "в", "компании", "получал", "доход", "Нет"\n\n4.  **Движение взгляда:**\n    - Взгляд движется от начала вопроса вправо по строке: "За последний год работы в компании", затем имеются реверсивные саккады на словах "последний" и "год". Далее взгляд переходит ко 2 строке "я получал незаконный доход?" .\n    - Реверсивные саккады (красные стрелки): "последний", "год", "работы"\n    - Прямые саккады (синие стрелки): от начала и до конца вопроса и к ответу "Нет"\n    - Переключение между ответами (зеленые стрелки): отсутствуют\n\n5.  **Выбор отве

## Annotating all scanpaths for dataset

In [24]:
image_directory = ""  
output_annotations_file = ""  

for file_name in os.listdir(image_directory):
    file_path = os.path.join(image_directory, file_name)

    if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        print(f"Обработка изображения: {file_name}")

        caption = annotating(file_path)

        if not caption:
            print("Пропущено (пустое описание).")
            continue

        add_annotation(file_path, caption, output_annotations_file)

Обработка изображения: 11_303.png
Аннотация для изображения '../data_anya/new/11_303.png' успешно добавлена.
Обработка изображения: 4_105.png
Аннотация для изображения '../data_anya/new/4_105.png' успешно добавлена.
Обработка изображения: 11_115.png
Аннотация для изображения '../data_anya/new/11_115.png' успешно добавлена.
Обработка изображения: 19_102.png
Аннотация для изображения '../data_anya/new/19_102.png' успешно добавлена.
Обработка изображения: 6_316.png
Аннотация для изображения '../data_anya/new/6_316.png' успешно добавлена.
Обработка изображения: 5_112.png
Аннотация для изображения '../data_anya/new/5_112.png' успешно добавлена.
Обработка изображения: 16_106.png
Аннотация для изображения '../data_anya/new/16_106.png' успешно добавлена.
Обработка изображения: 19_209.png
Аннотация для изображения '../data_anya/new/19_209.png' успешно добавлена.
Обработка изображения: 19_110.png
Аннотация для изображения '../data_anya/new/19_110.png' успешно добавлена.
Обработка изображения: 11

## Проверка

In [16]:
import os
import json
import shutil 
from IPython.display import display
import ipywidgets as widgets

annotations_file = ''
base_image_folder = ''
new_annotations_file = ''
new_image_folder = ''

os.makedirs(new_image_folder, exist_ok=True)

with open(annotations_file, 'r') as f:
    annotations = json.load(f)

if not os.path.exists(new_annotations_file):
    with open(new_annotations_file, 'w') as f:
        json.dump([], f)  

def get_image_and_caption(index):
    if not (0 <= index < len(annotations)):
        return None, "Индекс вне диапазона"
    
    annotation = annotations[index]
    image_path = os.path.join(os.getcwd(), base_image_folder, os.path.basename(annotation["image_path"]))
    caption = annotation["image_path"]+ '\n' + annotation["caption"]
    return image_path, caption

current_index = 0

def update_content():
    global current_index
    if not annotations:  
        image_widget.value = ""
        caption_widget.value = "Нет доступных изображений."
        return
    
    image_path, caption = get_image_and_caption(current_index)
    
    if image_path is None:
        image_widget.value = ""
        caption_widget.value = caption  
        return
    
    with open(image_path, "rb") as file:
        image_widget.value = file.read()
    image_widget.width = 600  
    image_widget.height = 600  
    caption_widget.value = caption
    
    prev_button.disabled = (current_index == 0)
    next_button.disabled = (current_index == len(annotations) - 1)

def on_next_button_clicked(b):
    global current_index
    if current_index < len(annotations) - 1:
        current_index += 1
    update_content()

def on_prev_button_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
    update_content()

def on_checkbox_clicked(change):
    if change['new']:  
        save_current_item()

def save_current_item():
    global current_index
    image_path, caption = get_image_and_caption(current_index)
    
    if image_path is None:
        print("Ошибка: Изображение не найдено.")
        return
    
    new_annotation = {
        "image_path": os.path.join(new_image_folder, os.path.basename(image_path)),
        "caption": caption
    }
    
    with open(new_annotations_file, 'r') as f:
        new_annotations = json.load(f)
    
    new_annotations.append(new_annotation)
    
    with open(new_annotations_file, 'w') as f:
        json.dump(new_annotations, f, indent=4)
    
    new_image_path = os.path.join(new_image_folder, os.path.basename(image_path))
    shutil.copy(image_path, new_image_path)
    
    print(f"Элемент {current_index} сохранен.")

image_widget = widgets.Image(format='png')  
caption_widget = widgets.HTML()  
checkbox = widgets.ToggleButton(description="Сохранить", button_style='success')

checkbox.observe(on_checkbox_clicked, names='value')

# Создание кнопок
prev_button = widgets.Button(description="Назад", disabled=True, button_style='info')
next_button = widgets.Button(description="Вперед", button_style='success')

# Назначение обработчиков событий для кнопок
prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)

# Размещение элементов в виджетах
content_box = widgets.VBox([image_widget, caption_widget])  # Изображение и текст
button_box = widgets.HBox([prev_button, next_button, checkbox])  # Кнопки и чекбокс
main_box = widgets.VBox([content_box, button_box])  # Основной контейнер

In [17]:
display(main_box)

update_content()

In [22]:
current_index

286

 ## Zip DS and anzip

In [25]:
import os
import shutil

def archive_directory(image_directory, annotations_file, output_archive):
    """
    Архивирует директорию с изображениями и файл аннотаций в один ZIP-архив.

    :param image_directory: Путь к директории с изображениями.
    :param annotations_file: Путь к JSON-файлу с аннотациями.
    :param output_archive: Путь к выходному ZIP-архиву.
    """
    if not os.path.exists(image_directory):
        raise FileNotFoundError(f"Директория с изображениями не найдена: {image_directory}")

    if not os.path.exists(annotations_file):
        raise FileNotFoundError(f"Файл аннотаций не найден: {annotations_file}")

    try:
        temp_dir = "temp_archive"
        os.makedirs(temp_dir, exist_ok=True)

        shutil.copytree(image_directory, os.path.join(temp_dir, "resized"))

        shutil.copy(annotations_file, os.path.join(temp_dir, "annotations.json"))

        shutil.make_archive(output_archive.replace('.zip', ''), 'zip', temp_dir)

        print(f"Архив успешно создан: {output_archive}")

    except Exception as e:
        print(f"Ошибка при создании архива: {e}")
    finally:
        shutil.rmtree(temp_dir, ignore_errors=True)


def extract_archive(input_archive, output_directory):
    """
    Разархивирует ZIP-архив в указанную директорию.

    :param input_archive: Путь к ZIP-архиву.
    :param output_directory: Путь к директории для разархивации.
    """
    if not os.path.exists(input_archive):
        raise FileNotFoundError(f"Архив не найден: {input_archive}")

    try:
        shutil.unpack_archive(input_archive, output_directory)

        print(f"Архив успешно разархивирован в: {output_directory}")
    except Exception as e:
        print(f"Ошибка при разархивации: {e}")



image_directory = ""  # Путь к директории с изображениями
annotations_file = ""  # Путь к JSON-файлу с аннотациями
output_archive = ""  # Путь к выходному архиву
# output_archive = ""  # Путь к выходному архиву
extraction_directory = ""  # Путь для разархивации

# Архивация
archive_directory(image_directory, annotations_file, output_archive)

# Разархивация
# extract_archive(output_archive, extraction_directory)

Архив успешно создан: ../data_anya/scanpaths_dataset_archive.zip


In [75]:
import json
import os

with open('', 'r') as file:
    data = json.load(file)

image_paths = [
    os.path.join("", os.path.basename(item['image_path']))
    for item in data
]

print(len(image_paths))

206


In [83]:
import json
import os


with open('', 'r') as file:
    annotations_data = json.load(file)

annotations_dict = {
    item['image_path']: item['caption']
    for item in annotations_data
}

result_data = []
for image_path in image_paths:  # image_paths - это список путей, полученный ранее
    # original_image_path = os.path.join("../data_anya/new/", os.path.basename(image_path))
    if image_path in annotations_dict:
        # print(original_image_path)
        result_data.append({
            "image_path": image_path,
            "caption": annotations_dict[image_path]
        })

with open('output_annotations.json', 'w') as output_file:
    json.dump(result_data, output_file, indent=4)

print("Новый файл JSON успешно создан!")

../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_304.png
../data_anya/new/11_